In [ ]:
# Notebook parameters. Values here are for development only and
# will be overridden when running via snakemake and papermill.
contig = "2RL"
cohorts_analysis = "20240924"
analysis_version = "2025.02.07"
dask_scheduler = "single-threaded"

In [ ]:
from pyprojroot import here

root = here()
%run -i {root}/workflow/common/scripts/setup.py
%run -i {root}/workflow/site/scripts/page-setup.py

# N.B., do not add the "remove-output" tag to this cell!!! If you do,
# the bokeh javascript libraries will not get loaded in the generated
# HTML page. The call to output_notebook() injects javascript in the
# cell output which triggers the bokeh javascript libraries to be loaded
# in the page.
output_notebook(hide_banner=True)

# Chromosome Foo

The plot below shows {term}`selection signals` discovered in the {term}`malaria vector` species *An. gambiae*, *An. coluzzii* or *An. arabiensis*, all of which are members of the {term}`*Anopheles gambiae* species complex`. Hover over a signal for more information about the {term}`cohort` in which the signal was found, including the mosquito species, collection location and date range. Click on a signal to see the underlying selection scan data.

In [ ]:
df_signals = load_signals(contig=contig)

plot_signals(df=df_signals, contig=contig)

In [ ]:
df_signals = df_signals[
    [
        "contig",
        "focus_pstart",
        "focus_pstop",
        "cohort_id",
        "cohort_label",
        "statistic",
        "delta_i",
    ]
]
df_signals = df_signals.assign(
    focal_region=df_signals["contig"]
    + " ( "
    + df_signals["focus_pstart"].apply(
        lambda x: "{:,}".format(
            x,
        )
    )
    + " - "
    + df_signals["focus_pstop"].apply(
        lambda x: "{:,}".format(
            x,
        )
    )
    + " )"
)


def make_clickable(url, name):
    return '<a href="{}" rel="noopener noreferrer" target="_blank">{}</a>'.format(
        url, name
    )


df_signals["url"] = "../cohort/" + df_signals["cohort_id"] + ".html"
df_signals["cohort_label"] = df_signals.apply(
    lambda x: make_clickable(x["url"], x["cohort_label"]), axis=1
)
df_signals[["focal_region", "cohort_label", "statistic", "delta_i"]].rename(
    columns={
        "focal_region": "Focal Region",
        "cohort_label": "Cohort Label",
        "statistic": "Statistic",
        "delta_i": "Delta i",
    }
).style.hide(axis="index")